## Analyze and visualize Ai map

In [ ]:
import dask as da
from dask import array
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import h5py
import avul
import importlib
from scipy import ndimage
import scipy.misc
import skimage.measure
import rasterio
import csv
from multiprocessing.pool import ThreadPool
from skimage.measure import block_reduce
from skimage.measure import regionprops

### Load and plot Ai map
Cell below loads the Ai map of the study area created in the "RunTiles.ipynb" notebook. This is then plotted. The array "CutActLevIm" can also be plotted as a way to visualize areas above a threshold Ai value.

The output of this cell was used in part to create figure 8a and 8b of the paper. For figure 8b a slightly modified version was used to zoom into the the high activity level area pictured.

In [ ]:
%matplotlib notebook
data = np.load('../Data/DeepWaterAndManualMasked365PxAltiPlanoStride60.npz')
ActLevIm = data['ActLevIm']
CutActLevIm = ActLevIm * (ActLevIm >= 12) #Display Ai values greater than or equal to a certain value
plt.imshow(CutActLevIm)
#plt.colorbar()
#plt.savefig("AiMapZoomedIn.png",dpi=350)

## Load test region and mask and compute windows
This is the same script that loads the test region and mask in "RunTiles.ipynb". Disregard the checking output this time around

In [ ]:
%run -i 'loadregionandmask.py'

### Get geotransform for test region
The first cell below is the geotransform for the full study area used in the paper. Run this cell if you are using the full study area. The second cell below is the geotransform for the small test area. Run the second cell if you are running the small test area.

This transform is used to get the geographical centers of the A_i regions that will be found later on in this notebook.

In [ ]:
left = -68.91852401882868
bottom = -17.576032607204688 
right = -64.50312473432442
top = -13.160633322700418
NewTran = rasterio.transform.from_bounds(left, bottom, right, top, 16384, 16384)

In [ ]:
left = -68.91852401882868
bottom = -14.633151736429138 
right = -67.44600560509997
top = -13.160633322700418
NewTran = rasterio.transform.from_bounds(left, bottom, right, top, 16384, 16384)

In [ ]:
from IPython.core.debugger import set_trace
from PIL import Image
#Write video of all the areas that have greater than or equal to a certain active level
#Can then watch this video for different sections to look for cutoffs and avulsions in the active level 1 regions
MaxActLevThresh = 12
ActLevThresh = 2
CutActLevIm = ActLevIm * (ActLevIm >= ActLevThresh)
LabCutIm,tra = ndimage.measurements.label(CutActLevIm,structure=np.ones((3,3)))
labels = np.unique(LabCutIm[np.where(LabCutIm)])
CurLevRng = []
CurLevPts = []
total_number_of_frames = 34
#empty array of region id's and latitude's and longitude's. Right now only write for active level 2
ActLevRegLocs = []
AllRegAreas = [{},{},{},{},{},{},{},{},{},{},{},{}]
#there needs to be the same number of active levels as there are areas
for y in range(labels.size):
    #initialize ghetto way to turn 3d numpy array into list of 2d arrays. 
    allFrames = []
    print("current label")
    print(y)
    #Binary 
    #set_trace()
    CurLabIm = LabCutIm * (LabCutIm == labels[y])
    CurLabIm[CurLabIm == np.max(CurLabIm)] = 1
    CurRegActIm = ActLevIm * CurLabIm
    
    #reset ActLevThresh for next region
    ActLevThresh = 2
    while CurRegActIm.any():
        #plt.imshow(CurLabIm)
        #for this analysis need to cut out bounding box to get all points in region      
        props = regionprops(CurLabIm.astype('int32'))
        maxLoc = tuple(np.subtract(props[0].bbox[2:4],1))
        minLoc = tuple(props[0].bbox[0:2])
        flRanMax = np.max(SubWins[maxLoc])
        flRanMin = np.min(SubWins[minLoc])
        RangesMaxMin = [np.unravel_index(flRanMax,WinShape),np.unravel_index(flRanMin,WinShape)]
        SubIm = Im[:,RangesMaxMin[1][0]:RangesMaxMin[0][0]+1,RangesMaxMin[1][1]:RangesMaxMin[0][1]+1].astype('uint8')
        #get center of region on the inside the actual surface water image
        #set_trace()
        regcent = (np.round(RangesMaxMin[1][0]+(SubIm.shape[1]/2)),np.round(RangesMaxMin[1][1]+(SubIm.shape[2]/2)))
        
        #get real world location
        longlat = rasterio.transform.xy(NewTran, regcent[0], regcent[1], offset='center')

        SubMask = mask[RangesMaxMin[1][0]:RangesMaxMin[0][0]+1,RangesMaxMin[1][1]:RangesMaxMin[0][1]+1].astype('uint8')
        #GetReg returns the fluvial components in the region 
        #set_trace()
        ComIm = GetReg(SubIm,SubMask)

        RegDim = ComIm.shape

        #write each layer in the image as a seperate image so you can view as a video in imagej
        for x in range(ComIm.shape[0]):
            fr = ComIm[x,:,:]
            allFrames.append(fr)

        # Now we can do the plotting!
        all_data = allFrames
        fig, ax = plt.subplots(1, figsize=(4, 4))
        # Remove a bunch of stuff to make sure we only 'see' the actual imshow
        # Stretch to fit the whole plane
        fig.subplots_adjust(0, 0, 1, 1)
        # Remove bounding line
        ax.axis("off")
        # Initialise our plot. Make sure you set vmin and vmax to the active level range!
        image = ax.imshow(all_data[0], vmin=0, vmax=1)
        animation = FuncAnimation(fig,animate,np.arange(total_number_of_frames),fargs=[],interval=1000 / 5)
        # Try to set the DPI to the actual number of pixels you're plotting
        #save line for saving on local computer
        #animation.save("/home/dylan/GlobalAvulsion/GlobalAvulsionData/TestRegionResults/AltiPlanocornerActLev"+ str(ActLevThresh) + 'Reg'+ str(y).zfill(2)+".mp4", dpi=512)
        #gcloud save line
        animation.save("./ActLev"+ str(ActLevThresh) + '/' + 'Reg'+ str(y).zfill(3)+"ActLev"+str(ActLevThresh)+
                       "row"+str(maxLoc[0])+"col"+str(maxLoc[1])+".avi", dpi=512)
        
        #append the region information to a list containing the region id + lat and long of that region
        #if ActLevThresh == 2:
        #    ActLevRegLocs.append('Reg'+ str(y).zfill(3)+"ActLev"+str(ActLevThresh) + " Latitude:" + str(longlat[1]) + " Longitude:" + str(longlat[0]))
        
        #write
        AllRegAreas[ActLevThresh-1][y] = RegDim[1]*RegDim[2]
        #raise threshold for next iteration of this region
        ActLevThresh = ActLevThresh + 1
        #set_trace()
        CurRegActIm = CurRegActIm * (CurRegActIm >= ActLevThresh)
        #print("maximum active level for region")
        #print(np.max(CurRegActIm))
        BinCurRegActIm = CurRegActIm.copy()
        BinCurRegActIm[BinCurRegActIm >= 2] = 1
        CurLabIm = BinCurRegActIm
        

In [ ]:
#code snippet to save the areas of all the regions you've recorded to a seperate csv file for each active level
for x in range(12):
    # define a dictionary with key value pairs
    CurAreadict = AllRegAreas[x]
    # open file for writing, "w" is writing
    myfile = open("ActLev" + str(x+1) + "areas.csv",'w')
    w = csv.writer(myfile)
    # loop over dictionary keys and values
    for key, val in CurAreadict.items():
        # write every key and value to file. Convert to units of km^2
        w.writerow([key, val*0.0009])
        myfile.flush()

In [ ]:
%matplotlib notebook
#plot the final results of the region analysis.
fig, (ax1, ax2, ax3) = plt.subplots(1,3,figsize=(12.5,4))
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
ax1.scatter(np.asarray([12,11,10,9,8,7,6,5,4,3,2]),np.asarray([1,1,1,1,1,.75,.73,.72,.75,.72,.58]))
ax1.set_xlim(12, 1)
ax1.set_xlabel('Threshold $A_i$',fontsize=14)
ax1.set_ylabel('fraction of regions containing event',fontsize=14)
ax1.tick_params(axis='both', labelsize=12)

ax2.scatter(np.asarray([12,11,10,9,8,7,6,5,4,3,2]),np.asarray([199,358,378,557,826,3397,4668,10516,18406,28955,54569]))
ax2.set_xlim(12, 1)
ax2.set_xlabel('Threshold $A_i$',fontsize=14)
ax2.set_ylabel('Total area of regions ($km^2$)',fontsize=14)
ax2.tick_params(axis='both', labelsize=12)

#plot event density per km^2 for the different active levels
ax3.scatter(np.asarray([12,11,10,9,8,7,6,5,4,3,2]),np.asarray([.015,.02,.019,.016,.017,.009,.008,.01,.007,.005,.005])*100)
ax3.set_xlabel('Threshold $A_i$',fontsize=14)
ax3.set_ylabel('events per 100 $km^2$',fontsize=14)
ax3.set_xlim(12, 1)
ax3.tick_params(axis='both', labelsize=12)

plt.tight_layout()
plt.savefig("RegionResultPlots.png",dpi=350)

In [ ]:
#plot an example of #of endtimes stabalizing as you increase the step length l. This is currently figure _
stepL = np.asarray([1.732,3.464,5.196,6.928,8.660,10.392,12.124,13.856,15.588,17.320,19.052,20.784,22.516])
Ai = np.asarray([35,35,31,30,28,24,17,14,10,10,10,10,10])
plt.scatter(stepL,Ai)
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
plt.ylabel('$A_i$',fontsize=14)
plt.xlabel('step length $l$',fontsize=14)
plt.tick_params(axis='both', labelsize=12)
plt.savefig("AiStablizingPlot.png",dpi=350)

In [ ]:
#code snippet for figure 8 panel c
%matplotlib inline
for x in range(35):
    colIm = ComIm[x]
    colIm[colIm==1] = x
    plt.imshow(SubIm[x],alpha=.1,cmap='viridis')
plt.savefig("Figure8PanelC.png",dpi=350)

In [ ]:
# code for figure 9
%matplotlib inline
CutActLevIm6 = ActLevIm * (ActLevIm >= 6)
CutActLevIm4 = ActLevIm * (ActLevIm >= 4)
CutActLevIm2 = ActLevIm * (ActLevIm >= 2)
fig, (ax1, ax2, ax3) = plt.subplots(1, 3)
ax1.imshow((CutActLevIm6 > 0).astype(int))
ax2.imshow((CutActLevIm4 > 0).astype(int))
ax3.imshow((CutActLevIm2 > 0).astype(int))
plt.tight_layout()
plt.savefig("Fig9PlotsUnlabeled.png",dpi=350)


#if you want to use a colorbar
#subplot(1, 3, 1)
#plt.imshow((LabCutIm > 0).astype(int),vmin=0, vmax=1, cmap='viridis', aspect='auto')
#plt.colorbar()